In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/DATASET_FIX/Training_resize /content/
!cp -r /content/drive/MyDrive/DATASET_FIX/Val_resize /content/

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Path untuk data pelatihan
train_dir = '/content/Training_resize'
val_dir = '/content/Val_resize'

# Augmentasi data untuk pelatihan
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

# Memuat data pelatihan
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=16,
    shuffle=True,
    seed=42
)

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=16,
    shuffle=True,
    seed=42
)

# Load model ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Menambahkan layer custom untuk klasifikasi
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Kompilasi model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

# Training model
epochs = 15
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[early_stopping, reduce_lr]
)


Found 3500 images belonging to 5 classes.
Found 700 images belonging to 5 classes.
94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/15
219/219 [==============================] - 91s 204ms/step - loss: 0.6230 - accuracy: 0.7703 - val_loss: 1.9836 - val_accuracy: 0.1957 - lr: 1.0000e-04
Epoch 2/15
219/219 [==============================] - 35s 161ms/step - loss: 0.2519 - accuracy: 0.9146 - val_loss: 1.7280 - val_accuracy: 0.1857 - lr: 1.0000e-04
Epoch 3/15
219/219 [==============================] - 37s 167ms/step - loss: 0.1497 - accuracy: 0.9529 - val_loss: 2.2637 - val_accuracy: 0.1814 - lr: 1.0000e-04
Epoch 4/15
219/219 [==============================] - 38s 172ms/step - loss: 0.1708 - accuracy: 0.9474 - val_loss: 2.6141 - val_accuracy: 0.4000 - lr: 1.0000e-04
Epoch 5/15
219/219 [==============================] - 37s 168ms/step - loss: 0.1157 - accuracy: 0.9654 - val_loss: 1.6299 - val_accuracy: 0.6186 - lr: 1.0000e-04
Epoch 6/15
219/219 [======================

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Path untuk data pelatihan
train_dir = '/content/Training_resize'
val_dir = '/content/Val_resize'

# Augmentasi data untuk pelatihan
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,  # Kurangi tingkat augmentasi
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
)

# Memuat data pelatihan
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,  # Coba ukuran batch yang lebih besar
    shuffle=True,
    seed=42
)

val_datagen = ImageDataGenerator(rescale=1./255)  # Hanya rescale untuk data validasi
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    shuffle=False,
    seed=42
)

# Load model ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Menambahkan layer custom untuk klasifikasi
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)  # Tambahkan BatchNormalization
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Kompilasi model
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

# Training model dengan lebih banyak epochs
epochs = 50
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[early_stopping, reduce_lr]
)


Found 3500 images belonging to 5 classes.
Found 700 images belonging to 5 classes.
Epoch 1/50
110/110 [==============================] - 70s 292ms/step - loss: 3.1050 - accuracy: 0.3651 - val_loss: 3.9140 - val_accuracy: 0.2000 - lr: 1.0000e-05
Epoch 2/50
 16/110 [===>..........................] - ETA: 26s - loss: 2.5023 - accuracy: 0.5781

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7bff57b2-cd1b-4d31-87b2-4f5483361437' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>